In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter


In [3]:
from pathlib import Path
p = Path('../data')

In [4]:
data_files = list(p.glob('**/*.txt'))

In [5]:
data_files

[PosixPath('../data/background.txt'),
 PosixPath('../data/noble_prize_intro.txt'),
 PosixPath('../data/nomination.txt'),
 PosixPath('../data/prizes.txt')]

In [25]:
documents = []
for file in data_files:
    loader = TextLoader(file)
    doc = loader.load()
    documents.append(doc[0])
        

In [31]:
print(documents[0].page_content)

Alfred Nobel, in his last will and testament, stated that his wealth should be used to create a series of prizes for those who confer the "greatest benefit on mankind" in the fields of physics, chemistry, peace, physiology or medicine, and literature.[3] Though Nobel wrote several wills during his lifetime, the last one was written a year before he died and was signed at the Swedish-Norwegian Club in Paris on 27 November 1895.[4][5] Nobel bequeathed 94% of his total assets, 31 million Swedish kronor ($2.9 million USD, or €2.7 million in 2023), to establish and endow the five Nobel Prizes.[6] Owing to the level of skepticism surrounding the will, it was not until 26 April 1897 that it was approved by the Storting (Norwegian Parliament).[7][8] The executors of his will were Ragnar Sohlman and Rudolf Lilljequist, who formed the Nobel Foundation to take care of Nobel's fortune and organise the prizes.
The members of the Norwegian Nobel Committee who were to award the Peace Prize were appoi

In [26]:
llm = ChatOpenAI()

In [27]:
embeddings = OpenAIEmbeddings()

In [32]:
vector = FAISS.from_documents(documents, embeddings)

In [33]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

In [34]:
document_chain = create_stuff_documents_chain(llm, prompt)


retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [36]:
response = retrieval_chain.invoke({"input": "When is nobel prize awarded"})
print(response["answer"])


The Nobel Prize is awarded annually in a formal ceremony held on December 10th, the anniversary of Alfred Nobel's death.
